<a href="https://colab.research.google.com/github/Eman-diab-hussein/deep/blob/main/lab_os_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile hello.cpp
#include <iostream>
#include <omp.h>
int main() {
    const int n = 1000;
    int array[n];
      // Set number of threads to 10
    omp_set_num_threads(10);
    // Initialize array in parallel
    #pragma omp parallel for schedule(static)
    for (int i = 0; i < n; i++)
    {    array[i] = i * i;
        std::cout << "Thread " << omp_get_thread_num() << " computed array[" << i << "] = " << array[i] << "\n";}
    // Sum the elements in parallel
    int sum = 0;
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < n; i++) {
        sum += array[i];
    }
    std::cout << "Total sum: " << sum << std::endl;
    return 0;}


Overwriting hello.cpp


In [ ]:
!g++ -fopenmp hello.cpp -o hello


In [ ]:
!./hello

Thread 3 computed array[300] = 90000
Thread 3 computed array[301] = 90601
Thread 3 computed array[302] = 91204
Thread 3 computed array[303] = 91809
Thread 3 computed array[304] = 92416
Thread 3 computed array[305] = 93025
Thread 3 computed array[306] = 93636
Thread 3 computed array[307] = 94249
Thread 3 computed array[308] = 94864
Thread 3 computed array[309] = 95481
Thread 3 computed array[310] = 96100
Thread Thread 3 computed array[4 computed array[400311] = ] = 160000
Thread 1 computed array[100] = 10000
Thread 5 computed array[500] = 250000
96721Thread 5 computed array[501] = 251001
Thread Thread 5 computed array[502] = 252004
Thread 0 computed array[0] = 0
Thread 4 computed array[401] = Thread 1608010 computed array[1

Thread 4 computed array[Thread 402] = 161604
Thread 4 computed array[403] = 162409
Thread 4 computed array[404] = 163216
Thread 4 computed array[405] = 164025
Thread 4 computed array[406] = 164836
Thread 4 computed array[407] = 165649
] = 1
3 computed array[312Threa

In [ ]:
%%writefile hello1.cpp
#include <omp.h>
#include<iostream>
static long num_steps = 100000;
double step;
int main ()
{
int i;
double x, pi, sum = 0.0;
 step = 1.0/(double) num_steps;
#pragma omp parallel for private(x) reduction(+:sum)
 for (i=0;i< num_steps; i++){
 x = (i+0.5)*step;
 sum = sum + 4.0/(1.0+x*x);
 }
 pi = step * sum;
std::cout<<pi<<std::endl;
return 0;
}


Writing hello1.cpp


In [ ]:
!g++ -fopenmp hello1.cpp -o hello1


In [ ]:
!./hello1


3.14159


In [ ]:
%%writefile hello2.cpp
#include <iostream>
#include <omp.h>
#include <vector>
#include <random>

// Import necessary standard library components
using std::cout;
using std::endl;
using std::vector;
using std::uniform_int_distribution;

int main() {
    // Define matrix dimensions
    const int N = 5;  // Adjust size as needed for testing
    const int M = 5;
    const int P = 5;
    int i, j, k, tmp;
    // Random number generator setup
    std::random_device rd;
    std::mt19937 gen(rd());  // Seed the random generator
    uniform_int_distribution<> dis(1, 10);  // Define range [1, 10]
    // Initialize matrices A, B, and C
    vector<vector<int>> A(N, vector<int>(M, 1));  // N x M matrix with all elements as 1
    vector<vector<int>> B(M, vector<int>(P, 1));  // M x P matrix with all elements as 1
    vector<vector<int>> C(N, vector<int>(P, 0));  // N x P result matrix initialized to 0

    // Fill matrices A and B with random values
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < M; j++) {
            A[i][j] = dis(gen);  // Random value for matrix A
        }
    }
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            B[i][j] = dis(gen);  // Random value for matrix B
        }
    }
    // Matrix multiplication with OpenMP parallelization
    #pragma omp parallel for private(i, j, k, tmp)
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < P; j++) {
            int tmp = 0;
            for (int k = 0; k < M; k++) {
                tmp += A[i][k] * B[k][j];
            }
            C[i][j] = tmp;
            cout << "Thread " << omp_get_thread_num() << " computed array[" << i << "] = " << C[i][j] << "\n";
        }
    }
    // Print result (optional for small matrices)
    cout << "Result matrix C[0][0]: " << C[0][0] << endl;
    return 0;
}


Writing hello2.cpp


In [ ]:
!g++ -fopenmp hello2.cpp -o hello2

In [ ]:
! ./hello2

Thread 1 computed array[3] = 116
Thread 1 computed array[3] = 165
Thread 1 computed array[3] = 146
Thread 1 computed array[3] = 146
Thread 1 computed array[3] = 181
Thread 1 computed array[4] = 103
Thread 1 computed array[4] = 145
Thread 1 computed array[4] = 163
Thread 1 computed array[4] = 117
Thread 1 computed array[4] = 161
Thread 0 computed array[0] = 147
Thread 0 computed array[0] = 178
Thread 0 computed array[0] = 190
Thread 0 computed array[0] = 175
Thread 0 computed array[0] = 218
Thread 0 computed array[1] = 90
Thread 0 computed array[1] = 173
Thread 0 computed array[1] = 178
Thread 0 computed array[1] = 133
Thread 0 computed array[1] = 192
Thread 0 computed array[2] = 106
Thread 0 computed array[2] = 146
Thread 0 computed array[2] = 147
Thread 0 computed array[2] = 122
Thread 0 computed array[2] = 148
Result matrix C[0][0]: 147


In [ ]:
%%writefile matrix_multiplication.cpp
#include <iostream>
#include <vector>
#include <random>
#include <omp.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Function to initialize a matrix with random values
void initializeMatrix(vector<vector<int>> &matrix, int rows, int cols) {
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<int> dist(1, 10); // Random values between 1 and 10

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            matrix[i][j] = dist(gen);
        }
    }
}

int main() {
    const int N = 500; // Change to 1000000 for larger test cases
    const int M = 500;
    const int P = 500;
    int num_threads = 100;  // Change according to test case
    string scheduler_type = "static";  // Change to "dynamic" as needed

    vector<vector<int>> A(N, vector<int>(M));
    vector<vector<int>> B(M, vector<int>(P));
    vector<vector<int>> C(N, vector<int>(P, 0));

    // Initialize matrices
    initializeMatrix(A, N, M);
    initializeMatrix(B, M, P);

    omp_set_num_threads(num_threads);

    auto start = high_resolution_clock::now();  // Start timing

    #pragma omp parallel for schedule(static) // Change to schedule(dynamic) if needed
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < P; j++) {
            for (int k = 0; k < M; k++) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }

    auto end = high_resolution_clock::now();  // End timing
    duration<double> elapsed_time = end - start;

    cout << "Execution Time: " << elapsed_time.count() << " seconds" << endl;

    return 0;
}


Writing matrix_multiplication.cpp


In [ ]:
!g++ -fopenmp matrix_multiplication.cpp -o matrix_multiplication


In [ ]:
!./matrix_multiplication


Execution Time: 2.25152 seconds
